In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# preprocessing
train_images = train_images / 255.
test_images = test_images / 255.

In [ ]:
noise_factor = 0.2
train_images_noisy = train_images + noise_factor * np.random.normal(size=train_images.shape) 
test_images_noisy = test_images + noise_factor * np.random.normal(size=test_images.shape) 

In [ ]:
fig, axs = plt.subplots(1, 10, figsize=(15, 3))
plt.figure(figsize=(20, 2))
for img, ax in zip(train_images_noisy[:10], axs.flat):
    ax.imshow(img, cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')
plt.show()

In [ ]:
class Autoencoder(tf.keras.models.Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
               tf.keras.layers.Flatten(),
               tf.keras.layers.Dense(latent_dim, activation='relu'),
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(784, activation='sigmoid'),
            tf.keras.layers.Reshape((28, 28))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

latent_dim = 64 
autoencoder = Autoencoder(latent_dim)
autoencoder.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

In [ ]:
autoencoder.fit(train_images_noisy, train_images,
                epochs=10,
                shuffle=True,
                validation_data=(test_images_noisy, test_images))

In [ ]:
encoded_imgs = autoencoder.encoder(test_images_noisy).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
n = 10
fig, axs = plt.subplots(3, n, figsize=(20, 5))
for i in range(n):
    axs[0][i].set_title("original + noise")
    axs[0][i].imshow(test_images_noisy[i], cmap='gray')
   
    axs[1][i].set_title("reconstructed")
    axs[1][i].imshow(decoded_imgs[i], cmap='gray')
    
    axs[2][i].set_title("original")
    axs[2][i].imshow(test_images[i], cmap='gray')
    
for ax in fig.get_axes():
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')

plt.show()